# Scenario 2 Demo: Suburban RTK Evaluation

This notebook demonstrates the GNSS RTK evaluation pipeline for **scenario2** (Suburban environment).

**Steps:**
1. Validate scenario data with `--strict-real`
2. Run RTK evaluation with intent scoring
3. Display comparison results as a table
4. Plot baseline vs optimised horizontal error

In [ ]:
import subprocess
import os
import csv
from pathlib import Path

# Navigate to project root
PROJECT_ROOT = Path(os.getcwd()).parent.parent
GNSS_DIR = PROJECT_ROOT / "CODE" / "gnss"
SCENARIO = "scenario2"

print(f"Project root: {PROJECT_ROOT}")
print(f"GNSS module: {GNSS_DIR}")

## Step 1: Validate Scenario Data (Strict Real Mode)

**Note:** Scenario2 may fail strict validation if real RINEX data is not yet populated.

In [ ]:
# Run validate_scenario.py --strict-real scenario2
validate_cmd = [
    "python3", str(GNSS_DIR / "validate_scenario.py"),
    "--scenario", SCENARIO,
    "--strict-real"
]

print(f"Running: {' '.join(validate_cmd)}")
print("-" * 60)

result = subprocess.run(validate_cmd, cwd=str(GNSS_DIR), capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")
print("VALIDATION:", "PASSED" if result.returncode == 0 else "FAILED (may need real RINEX data)")

## Step 2: Run RTK Evaluation with Intent Scoring

In [ ]:
# Run rtk_evaluate.py --scenario scenario2 --intent accuracy
eval_cmd = [
    "python3", str(GNSS_DIR / "rtk_evaluate.py"),
    "--scenario", SCENARIO,
    "--intent", "accuracy"
]

print(f"Running: {' '.join(eval_cmd)}")
print("-" * 60)

result = subprocess.run(eval_cmd, cwd=str(GNSS_DIR), capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")

## Step 3: Load and Display comparison.csv

In [ ]:
# Find the latest evaluation output directory
output_root = PROJECT_ROOT / "OUTPUTs" / SCENARIO / "evaluation"

if output_root.exists():
    eval_dirs = sorted([d for d in output_root.iterdir() if d.is_dir()], reverse=True)
    if eval_dirs:
        latest_eval = eval_dirs[0]
        comparison_csv = latest_eval / "comparison.csv"
        print(f"Latest evaluation: {latest_eval.name}")
        print(f"Comparison CSV: {comparison_csv}")
    else:
        comparison_csv = None
        print("No evaluation directories found")
else:
    comparison_csv = None
    print(f"Output directory not found: {output_root}")

In [ ]:
# Read and display comparison.csv as a table (using csv module)
comparison_data = {}

if comparison_csv and comparison_csv.exists():
    with open(comparison_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            comparison_data = dict(row)
            break
    
    print("=" * 70)
    print(f"SCENARIO: {comparison_data.get('scenario', 'N/A')}")
    print(f"Timestamp: {comparison_data.get('timestamp', 'N/A')}")
    print(f"Simulated: {comparison_data.get('simulated', 'N/A')}")
    print(f"Intent: {comparison_data.get('intent', 'N/A')}")
    print(f"Intent Score: {comparison_data.get('intent_score', 'N/A')}")
    print("=" * 70)
    print()
    print(f"{'Metric':<35} {'Baseline':>15} {'Optimised':>15}")
    print("-" * 65)
    
    metrics = [
        ("horizontal_error_rms_m", "HPE RMS (m)"),
        ("horizontal_error_p95_m", "HPE P95 (m)"),
        ("vertical_error_rms_m", "VPE RMS (m)"),
        ("fix_rate_pct", "Fix Rate (%)"),
        ("availability_pct", "Availability (%)"),
    ]
    
    for metric_key, metric_name in metrics:
        baseline_val = comparison_data.get(f"baseline_{metric_key}", "")
        optimised_val = comparison_data.get(f"optimised_{metric_key}", "")
        print(f"{metric_name:<35} {baseline_val:>15} {optimised_val:>15}")
else:
    print("comparison.csv not found. Run evaluation first.")

## Step 4: Plot Baseline vs Optimised HPE RMS

In [ ]:
import matplotlib.pyplot as plt

if comparison_data:
    baseline_hpe = comparison_data.get("baseline_horizontal_error_rms_m", "")
    optimised_hpe = comparison_data.get("optimised_horizontal_error_rms_m", "")
    
    try:
        baseline_val = float(baseline_hpe) if baseline_hpe else 0.0
        optimised_val = float(optimised_hpe) if optimised_hpe else 0.0
    except ValueError:
        baseline_val, optimised_val = 0.0, 0.0
    
    fig, ax = plt.subplots(figsize=(8, 5))
    bars = ax.bar(["Baseline", "Optimised"], [baseline_val, optimised_val],
                  color=["#e74c3c", "#27ae60"], edgecolor="black", linewidth=1.2)
    
    for bar, val in zip(bars, [baseline_val, optimised_val]):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
                f"{val:.4f}" if val > 0 else "N/A",
                ha="center", va="bottom", fontsize=12, fontweight="bold")
    
    ax.set_ylabel("Horizontal Position Error RMS (m)", fontsize=12)
    ax.set_title(f"Scenario 2: Baseline vs Optimised HPE RMS\nIntent: {comparison_data.get('intent', 'N/A')}", fontsize=14)
    ax.set_ylim(0, max(baseline_val, optimised_val, 0.1) * 1.3)
    ax.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("No comparison data available for plotting.")

---

**Output Files:**
- `OUTPUTS/scenario2/evaluation/<timestamp>/comparison.csv`
- `OUTPUTS/scenario2/evaluation/<timestamp>/comparison.json`
- `OUTPUTS/scenario2/evaluation/<timestamp>/intent_score_explain.json`

# Scenario 2 Demo: Suburban RTK Evaluation

This notebook demonstrates the GNSS RTK evaluation pipeline for **scenario2** (Suburban environment).

**Steps:**
1. Validate scenario data with `--strict-real`
2. Run RTK evaluation with intent scoring
3. Display comparison results as a table
4. Plot baseline vs optimised horizontal error

In [ ]:
import subprocess
import os
import csv
from pathlib import Path

# Navigate to project root (adjust if running from different location)
PROJECT_ROOT = Path(os.getcwd()).parent.parent
GNSS_DIR = PROJECT_ROOT / "CODE" / "gnss"
SCENARIO = "scenario2"

print(f"Project root: {PROJECT_ROOT}")
print(f"GNSS module: {GNSS_DIR}")

## Step 1: Validate Scenario Data (Strict Real Mode)

**Note:** Scenario2 may fail strict validation if real RINEX data is not yet populated.

In [ ]:
# Run validate_scenario.py --strict-real scenario2
validate_cmd = [
    "python3", str(GNSS_DIR / "validate_scenario.py"),
    "--scenario", SCENARIO,
    "--strict-real"
]

print(f"Running: {' '.join(validate_cmd)}")
print("-" * 60)

result = subprocess.run(
    validate_cmd,
    cwd=str(GNSS_DIR),
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")
validation_passed = result.returncode == 0
print("VALIDATION:", "PASSED" if validation_passed else "FAILED (may need real RINEX data)")

## Step 2: Run RTK Evaluation with Intent Scoring

In [ ]:
# Run rtk_evaluate.py --scenario scenario2 --intent accuracy
eval_cmd = [
    "python3", str(GNSS_DIR / "rtk_evaluate.py"),
    "--scenario", SCENARIO,
    "--intent", "accuracy"
]

print(f"Running: {' '.join(eval_cmd)}")
print("-" * 60)

result = subprocess.run(
    eval_cmd,
    cwd=str(GNSS_DIR),
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")

## Step 3: Load and Display comparison.csv

In [ ]:
# Find the latest evaluation output directory
output_root = PROJECT_ROOT / "OUTPUTs" / SCENARIO / "evaluation"

if output_root.exists():
    eval_dirs = sorted([d for d in output_root.iterdir() if d.is_dir()], reverse=True)
    if eval_dirs:
        latest_eval = eval_dirs[0]
        comparison_csv = latest_eval / "comparison.csv"
        print(f"Latest evaluation: {latest_eval.name}")
        print(f"Comparison CSV: {comparison_csv}")
    else:
        print("No evaluation directories found")
        comparison_csv = None
else:
    print(f"Output directory not found: {output_root}")
    print("Run Step 2 first to generate outputs.")
    comparison_csv = None

In [ ]:
# Read and display comparison.csv as a table
# Using csv module (no external dependencies)

comparison_data = {}

if comparison_csv and comparison_csv.exists():
    with open(comparison_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            comparison_data = dict(row)
            break  # Single row
    
    # Display key metrics in a formatted table
    print("=" * 70)
    print(f"SCENARIO: {comparison_data.get('scenario', 'N/A')}")
    print(f"Timestamp: {comparison_data.get('timestamp', 'N/A')}")
    print(f"Simulated: {comparison_data.get('simulated', 'N/A')}")
    print(f"Intent: {comparison_data.get('intent', 'N/A')}")
    print(f"Intent Score: {comparison_data.get('intent_score', 'N/A')}")
    print("=" * 70)
    print()
    print(f"{'Metric':<35} {'Baseline':>15} {'Optimised':>15} {'Delta':>15}")
    print("-" * 80)
    
    metrics = [
        ("horizontal_error_rms_m", "HPE RMS (m)"),
        ("horizontal_error_p95_m", "HPE P95 (m)"),
        ("vertical_error_rms_m", "VPE RMS (m)"),
        ("fix_rate_pct", "Fix Rate (%)"),
        ("availability_pct", "Availability (%)"),
        ("ttff_sec", "TTFF (sec)"),
    ]
    
    for metric_key, metric_name in metrics:
        baseline_val = comparison_data.get(f"baseline_{metric_key}", "")
        optimised_val = comparison_data.get(f"optimised_{metric_key}", "")
        delta_key = metric_key.replace("_m", "_delta_m").replace("_pct", "_delta_pct").replace("_sec", "_delta_sec")
        delta_val = comparison_data.get(f"delta_{delta_key}", "")
        print(f"{metric_name:<35} {baseline_val:>15} {optimised_val:>15} {delta_val:>15}")
else:
    print("comparison.csv not found. Run evaluation first.")

## Step 4: Plot Baseline vs Optimised HPE RMS

In [ ]:
import matplotlib.pyplot as plt

if comparison_data:
    # Extract HPE RMS values
    baseline_hpe = comparison_data.get("baseline_horizontal_error_rms_m", "")
    optimised_hpe = comparison_data.get("optimised_horizontal_error_rms_m", "")
    
    # Convert to float (handle empty strings)
    try:
        baseline_val = float(baseline_hpe) if baseline_hpe else 0.0
        optimised_val = float(optimised_hpe) if optimised_hpe else 0.0
    except ValueError:
        baseline_val = 0.0
        optimised_val = 0.0
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=(8, 5))
    
    bars = ax.bar(
        ["Baseline", "Optimised"],
        [baseline_val, optimised_val],
        color=["#e74c3c", "#27ae60"],
        edgecolor="black",
        linewidth=1.2
    )
    
    # Add value labels on bars
    for bar, val in zip(bars, [baseline_val, optimised_val]):
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.01,
            f"{val:.4f}" if val > 0 else "N/A",
            ha="center", va="bottom", fontsize=12, fontweight="bold"
        )
    
    ax.set_ylabel("Horizontal Position Error RMS (m)", fontsize=12)
    ax.set_title(f"Scenario 2 (Suburban): Baseline vs Optimised HPE RMS\nIntent: {comparison_data.get('intent', 'N/A')}", fontsize=14)
    ax.set_ylim(0, max(baseline_val, optimised_val, 0.1) * 1.3)
    ax.grid(axis="y", linestyle="--", alpha=0.7)
    
    plt.tight_layout()
    plt.show()
else:
    print("No comparison data available for plotting.")

---

**Output Files Location:**
- `OUTPUTS/scenario2/evaluation/<eval_timestamp>/comparison.csv`
- `OUTPUTS/scenario2/evaluation/<eval_timestamp>/comparison.json`
- `OUTPUTS/scenario2/evaluation/<eval_timestamp>/intent_score_explain.json`

# Scenario 2 Demo: Suburban RTK Evaluation

This notebook demonstrates the GNSS RTK evaluation pipeline for **scenario2** (Suburban environment).

**Steps:**
1. Validate scenario data with `--strict-real`
2. Run RTK evaluation with intent scoring
3. Display comparison results as a table
4. Plot baseline vs optimised horizontal error

In [ ]:
import subprocess
import os
import csv
from pathlib import Path

# Navigate to project root (adjust if running from different location)
PROJECT_ROOT = Path(os.getcwd()).parent.parent
GNSS_DIR = PROJECT_ROOT / "CODE" / "gnss"
SCENARIO = "scenario2"

print(f"Project root: {PROJECT_ROOT}")
print(f"GNSS module: {GNSS_DIR}")

## Step 1: Validate Scenario Data (Strict Real Mode)

**Note:** Scenario2 may fail strict validation if real RINEX data is not yet populated.

In [ ]:
# Run validate_scenario.py --strict-real scenario2
validate_cmd = [
    "python3", str(GNSS_DIR / "validate_scenario.py"),
    "--scenario", SCENARIO,
    "--strict-real"
]

print(f"Running: {' '.join(validate_cmd)}")
print("-" * 60)

result = subprocess.run(
    validate_cmd,
    cwd=str(GNSS_DIR),
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")
validation_passed = result.returncode == 0
print("VALIDATION:", "PASSED" if validation_passed else "FAILED (may need real RINEX data)")

## Step 2: Run RTK Evaluation with Intent Scoring

In [ ]:
# Run rtk_evaluate.py --scenario scenario2 --intent accuracy
eval_cmd = [
    "python3", str(GNSS_DIR / "rtk_evaluate.py"),
    "--scenario", SCENARIO,
    "--intent", "accuracy"
]

print(f"Running: {' '.join(eval_cmd)}")
print("-" * 60)

result = subprocess.run(
    eval_cmd,
    cwd=str(GNSS_DIR),
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")

## Step 3: Load and Display comparison.csv

In [ ]:
# Find the latest evaluation output directory
output_root = PROJECT_ROOT / "OUTPUTs" / SCENARIO / "evaluation"

if output_root.exists():
    eval_dirs = sorted([d for d in output_root.iterdir() if d.is_dir()], reverse=True)
    if eval_dirs:
        latest_eval = eval_dirs[0]
        comparison_csv = latest_eval / "comparison.csv"
        print(f"Latest evaluation: {latest_eval.name}")
        print(f"Comparison CSV: {comparison_csv}")
    else:
        print("No evaluation directories found")
        comparison_csv = None
else:
    print(f"Output directory not found: {output_root}")
    print("Run Step 2 first to generate outputs.")
    comparison_csv = None

In [ ]:
# Read and display comparison.csv as a table
# Using csv module (no external dependencies)

comparison_data = {}

if comparison_csv and comparison_csv.exists():
    with open(comparison_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            comparison_data = dict(row)
            break  # Single row
    
    # Display key metrics in a formatted table
    print("=" * 70)
    print(f"SCENARIO: {comparison_data.get('scenario', 'N/A')}")
    print(f"Timestamp: {comparison_data.get('timestamp', 'N/A')}")
    print(f"Simulated: {comparison_data.get('simulated', 'N/A')}")
    print(f"Intent: {comparison_data.get('intent', 'N/A')}")
    print(f"Intent Score: {comparison_data.get('intent_score', 'N/A')}")
    print("=" * 70)
    print()
    print(f"{'Metric':<35} {'Baseline':>15} {'Optimised':>15} {'Delta':>15}")
    print("-" * 80)
    
    metrics = [
        ("horizontal_error_rms_m", "HPE RMS (m)"),
        ("horizontal_error_p95_m", "HPE P95 (m)"),
        ("vertical_error_rms_m", "VPE RMS (m)"),
        ("fix_rate_pct", "Fix Rate (%)"),
        ("availability_pct", "Availability (%)"),
        ("ttff_sec", "TTFF (sec)"),
    ]
    
    for metric_key, metric_name in metrics:
        baseline_val = comparison_data.get(f"baseline_{metric_key}", "")
        optimised_val = comparison_data.get(f"optimised_{metric_key}", "")
        delta_key = metric_key.replace("_m", "_delta_m").replace("_pct", "_delta_pct").replace("_sec", "_delta_sec")
        delta_val = comparison_data.get(f"delta_{delta_key}", "")
        print(f"{metric_name:<35} {baseline_val:>15} {optimised_val:>15} {delta_val:>15}")
else:
    print("comparison.csv not found. Run evaluation first.")

## Step 4: Plot Baseline vs Optimised HPE RMS

In [ ]:
import matplotlib.pyplot as plt

if comparison_data:
    # Extract HPE RMS values
    baseline_hpe = comparison_data.get("baseline_horizontal_error_rms_m", "")
    optimised_hpe = comparison_data.get("optimised_horizontal_error_rms_m", "")
    
    # Convert to float (handle empty strings)
    try:
        baseline_val = float(baseline_hpe) if baseline_hpe else 0.0
        optimised_val = float(optimised_hpe) if optimised_hpe else 0.0
    except ValueError:
        baseline_val = 0.0
        optimised_val = 0.0
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=(8, 5))
    
    bars = ax.bar(
        ["Baseline", "Optimised"],
        [baseline_val, optimised_val],
        color=["#e74c3c", "#27ae60"],
        edgecolor="black",
        linewidth=1.2
    )
    
    # Add value labels on bars
    for bar, val in zip(bars, [baseline_val, optimised_val]):
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.01,
            f"{val:.4f}" if val > 0 else "N/A",
            ha="center", va="bottom", fontsize=12, fontweight="bold"
        )
    
    ax.set_ylabel("Horizontal Position Error RMS (m)", fontsize=12)
    ax.set_title(f"Scenario 2 (Suburban): Baseline vs Optimised HPE RMS\nIntent: {comparison_data.get('intent', 'N/A')}", fontsize=14)
    ax.set_ylim(0, max(baseline_val, optimised_val, 0.1) * 1.3)
    ax.grid(axis="y", linestyle="--", alpha=0.7)
    
    plt.tight_layout()
    plt.show()
else:
    print("No comparison data available for plotting.")

---

**Output Files Location:**
- `OUTPUTS/scenario2/evaluation/<eval_timestamp>/comparison.csv`
- `OUTPUTS/scenario2/evaluation/<eval_timestamp>/comparison.json`
- `OUTPUTS/scenario2/evaluation/<eval_timestamp>/intent_score_explain.json`